In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, nltk, warnings
import matplotlib.cm as cm
import itertools
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import preprocessing, model_selection, metrics, feature_selection
from sklearn.model_selection import GridSearchCV, learning_curve
from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix
from sklearn import neighbors, linear_model, svm, tree, ensemble
#from wordcloud import WordCloud, STOPWORDS
from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import PCA
from IPython.display import display, HTML
#import plotly.graph_objs as go
#from plotly.offline import init_notebook_mode,iplot
#init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
%matplotlib inline


In [2]:
train_set = pd.read_csv('Train_aBjfeNk.csv')
test_set = pd.read_csv('Test_LqhgPWU.csv')

In [3]:
train_set.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [4]:
train_set.describe(include = 'all')

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
count,8068.000000,8068,7928,8068.000000,7990,7944,7239.000000,8068,7733.000000,7992,8068
unique,NaN,2,2,NaN,2,9,NaN,3,NaN,7,4
top,NaN,Male,Yes,NaN,Yes,Artist,NaN,Low,NaN,Cat_6,D
freq,NaN,4417,4643,NaN,4968,2516,NaN,4878,NaN,5238,2268
mean,463479.214551,NaN,NaN,43.466906,NaN,NaN,2.641663,NaN,2.850123,NaN,NaN
std,2595.381232,NaN,NaN,16.711696,NaN,NaN,3.406763,NaN,1.531413,NaN,NaN
min,458982.000000,NaN,NaN,18.000000,NaN,NaN,0.000000,NaN,1.000000,NaN,NaN
25%,461240.750000,NaN,NaN,30.000000,NaN,NaN,0.000000,NaN,2.000000,NaN,NaN
50%,463472.500000,NaN,NaN,40.000000,NaN,NaN,1.000000,NaN,3.000000,NaN,NaN
75%,465744.250000,NaN,NaN,53.000000,NaN,NaN,4.000000,NaN,4.000000,NaN,NaN


In [5]:
train_set.describe()

,ID,Age,Work_Experience,Family_Size
count,8068.000000,8068.000000,7239.000000,7733.000000
mean,463479.214551,43.466906,2.641663,2.850123
std,2595.381232,16.711696,3.406763,1.531413
min,458982.000000,18.000000,0.000000,1.000000
25%,461240.750000,30.000000,0.000000,2.000000
50%,463472.500000,40.000000,1.000000,3.000000
75%,465744.250000,53.000000,4.000000,4.000000
max,467974.000000,89.000000,14.000000,9.000000


In [70]:
train_set.loc[train_set['Var_1'] == 'Cat_3']['Segmentation'].value_counts()

D    259
A    223
B    186
C    154
Name: Segmentation, dtype: int64

 train_setFamily_Size = train_setFamily_Sizefillna(2)

In [8]:
train_set['Work_Experience'] = train_set['Work_Experience'].fillna(3)

In [9]:
data1 = train_set.copy(deep = True)

In [10]:
data1['Var_1'] = data1['Var_1'].fillna('Cat_6')

In [11]:
data1.dropna(axis = 0 , inplace = True)

In [12]:
data1.describe(include= 'all')

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
count,7741.000000,7741,7741,7741.000000,7741,7741,7741.000000,7741,7741.000000,7741,7741
unique,NaN,2,2,NaN,2,9,NaN,3,NaN,7,4
top,NaN,Male,Yes,NaN,Yes,Artist,NaN,Low,NaN,Cat_6,D
freq,NaN,4251,4535,NaN,4850,2473,NaN,4692,NaN,5160,2148
mean,463470.630280,NaN,NaN,43.509237,NaN,NaN,2.672394,NaN,2.813719,NaN,NaN
std,2591.023084,NaN,NaN,16.697563,NaN,NaN,3.235406,NaN,1.505176,NaN,NaN
min,458982.000000,NaN,NaN,18.000000,NaN,NaN,0.000000,NaN,1.000000,NaN,NaN
25%,461260.000000,NaN,NaN,31.000000,NaN,NaN,0.000000,NaN,2.000000,NaN,NaN
50%,463450.000000,NaN,NaN,40.000000,NaN,NaN,1.000000,NaN,2.000000,NaN,NaN
75%,465714.000000,NaN,NaN,53.000000,NaN,NaN,4.000000,NaN,4.000000,NaN,NaN


In [13]:
data1.drop_duplicates(inplace = True)

In [14]:
data1.shape

(7741, 11)

In [15]:
data1['Gender'] = data1['Gender'].map({'Male':1,'Female':0})

In [16]:
data1.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,1,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,0,Yes,38,Yes,Engineer,3.0,Average,3.0,Cat_4,A
2,466315,0,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,1,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,0,Yes,40,Yes,Entertainment,3.0,High,6.0,Cat_6,A


In [17]:
data1['Graduated'] = data1['Graduated'].map({'Yes':1,'No':0})

In [18]:
data1['Ever_Married'] = data1['Ever_Married'].map({'Yes':1 , 'No': 0})

In [19]:
data1['Spending_Score'] = data1['Spending_Score'].map({'Low':1,'Average':2,'High':3})


In [20]:
data1['Var_1'] = data1['Var_1'].map({'Cat_1':1,'Cat_2':2,'Cat_3':3,'Cat_4':4,'Cat_5':5 , 'Cat_6':6 , 'Cat_7':7})


In [21]:
data1['Profession'].value_counts()

Artist           2473
Healthcare       1286
Entertainment     925
Engineer          679
Doctor            673
Lawyer            604
Executive         583
Marketing         280
Homemaker         238
Name: Profession, dtype: int64

In [22]:

data1['Profession'] = data1['Profession'].map({'Artist':1 , 'Healthcare':2 , 'Entertainment':3 , 'Engineer':4 , 'Doctor':5 , 'Lawyer':6 , 'Executive':7 , 'Marketing':8 , 'Homemaker':9})

In [23]:
data1['X_factor'] = data1['Spending_Score']/data1['Family_Size']

In [24]:
data1.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation,X_factor
0,462809,1,0,22,0,2,1.0,1,4.0,4,D,0.250000
1,462643,0,1,38,1,4,3.0,2,3.0,4,A,0.666667
2,466315,0,1,67,1,4,1.0,1,1.0,6,B,1.000000
3,461735,1,1,67,1,6,0.0,3,2.0,6,B,1.500000
4,462669,0,1,40,1,3,3.0,3,6.0,6,A,0.500000


In [25]:
X = data1.drop(['Segmentation'] , axis = 1)

In [26]:
Y = data1['Segmentation']

In [75]:
X.head()
X = X.drop(['Spending_Score','Family_Size'],axis = 1)

In [30]:
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier 
from sklearn.tree import DecisionTreeClassifier
#param_grid = {'criterion' : ['entropy', 'gini'], 'n_estimators' : [20, 40, 60, 80, 100],
               #'max_features' :['sqrt', 'log2']}
rfr = RandomForestClassifier(n_estimators = 100  , random_state = 1)
#rf.grid_search(parameters = param_grid, Kfold = 5)
#rfr_fit(X = X,Y = Y)
#rf.grid_predict(X_test, Y_test)
#gbc = GradientBoostingClassifier(random_state = 0)
#ada = AdaBoostClassifier(learning_rate = 0.01 , random_state = 0)
dtc = DecisionTreeClassifier(random_state = 0)
from sklearn.neighbors import KNeighborsClassifier

In [31]:
X.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,X_factor
0,462809,1,0,2.2,0,2,1.0,1,4.0,4,0.250000
1,462643,0,1,3.8,1,4,3.0,2,3.0,4,0.666667
2,466315,0,1,6.7,1,4,1.0,1,1.0,6,1.000000
3,461735,1,1,6.7,1,6,0.0,3,2.0,6,1.500000
4,462669,0,1,4.0,1,3,3.0,3,6.0,6,0.500000


In [33]:
X['Age'] *=10

In [34]:
rfr.fit(X,Y)
#knn.fit(X,Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [53]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import mean_squared_log_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report
import seaborn as sns
from collections import Counter

In [87]:
params = {}
params['learning_rate'] = 0.04
params['max_depth'] = 18
params['n_estimators'] = 3000
params['objective'] = 'multiclass'
params['boosting_type'] = 'gbdt'
params['subsample'] = 0.7
params['random_state'] = 42
params['colsample_bytree']=0.7
params['min_data_in_leaf'] = 55
params['reg_alpha'] = 1.7
params['reg_lambda'] = 1.11
params['class_weight']: {'A': 0.25, 'B': 0.20, 'C': 0.25, 'D' : 0.35}

In [86]:
Y.value_counts()

D    2148
C    1916
A    1886
B    1791
Name: Segmentation, dtype: int64

In [88]:
clf = lgb.LGBMClassifier(**params)

In [89]:
clf.fit(X,Y, verbose=True,)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
        importance_type='split', learning_rate=0.04, max_depth=18,
        min_child_samples=20, min_child_weight=0.001, min_data_in_leaf=55,
        min_split_gain=0.0, n_estimators=3000, n_jobs=-1, num_leaves=31,
        objective='multiclass', random_state=42, reg_alpha=1.7,
        reg_lambda=1.11, silent=True, subsample=0.7,
        subsample_for_bin=200000, subsample_freq=0)

In [90]:
clf.score(X,Y)

0.6780777677302674

In [35]:
test_set.describe(include = 'all')

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
count,2627.000000,2627,2577,2627.000000,2603,2589,2358.000000,2627,2514.000000,2595
unique,NaN,2,2,NaN,2,9,NaN,3,NaN,7
top,NaN,Male,Yes,NaN,Yes,Artist,NaN,Low,NaN,Cat_6
freq,NaN,1424,1520,NaN,1602,802,NaN,1616,NaN,1672
mean,463433.918919,NaN,NaN,43.649791,NaN,NaN,2.552587,NaN,2.825378,NaN
std,2618.245698,NaN,NaN,16.967015,NaN,NaN,3.341094,NaN,1.551906,NaN
min,458989.000000,NaN,NaN,18.000000,NaN,NaN,0.000000,NaN,1.000000,NaN
25%,461162.500000,NaN,NaN,30.000000,NaN,NaN,0.000000,NaN,2.000000,NaN
50%,463379.000000,NaN,NaN,41.000000,NaN,NaN,1.000000,NaN,2.000000,NaN
75%,465696.000000,NaN,NaN,53.000000,NaN,NaN,4.000000,NaN,4.000000,NaN


In [36]:
test_set['Family_Size'] = test_set['Family_Size'].fillna(2)

In [37]:
test_set['Work_Experience'] = test_set['Work_Experience'].fillna(3)

In [38]:
data2 = test_set.copy(deep = True)

In [39]:
data2['Var_1'] = data2['Var_1'].fillna('Cat_6')

In [40]:
#data2.dropna(axis = 0 , inplace = True)

In [41]:
data2['Gender'] = data2['Gender'].map({'Male':1,'Female':0})
data2['Graduated'] = data2['Graduated'].map({'Yes':1,'No':0})
data2['Ever_Married'] = data2['Ever_Married'].map({'Yes':1 , 'No': 0})
data2['Spending_Score'] = data2['Spending_Score'].map({'Low':1,'Average':2,'High':3})
data2['Var_1'] = data2['Var_1'].map({'Cat_1':1,'Cat_2':2,'Cat_3':3,'Cat_4':4,'Cat_5':5 , 'Cat_6':6 , 'Cat_7':7})
data2['Profession'] = data2['Profession'].map({'Artist':1 , 'Healthcare':2 , 'Entertainment':3 , 'Engineer':4 , 'Doctor':5 , 'Lawyer':6 , 'Executive':7 , 'Marketing':8 , 'Homemaker':9})

In [42]:
data2['Ever_Married'] = data2['Ever_Married'].fillna(1)
data2['Graduated'] = data2['Graduated'].fillna(1)

data2['Profession'] = data2['Profession'].fillna(1)

In [43]:
data2['X_factor'] = data2['Spending_Score']/data2['Family_Size']
#data2= data2.drop(['Spending_Score','Family_Size'],axis = 1)
data2.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,X_factor
0,458989,0,1.0,36,1.0,4.0,0.0,1,1.0,6,1.00
1,458994,1,1.0,37,1.0,2.0,8.0,2,4.0,6,0.50
2,458996,0,1.0,69,0.0,1.0,0.0,1,1.0,6,1.00
3,459000,1,1.0,59,0.0,7.0,11.0,3,2.0,6,1.50
4,459001,0,0.0,19,0.0,8.0,3.0,1,4.0,6,0.25


In [45]:
X.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,X_factor
0,462809,1,0,2.2,0,2,1.0,1,4.0,4,0.250000
1,462643,0,1,3.8,1,4,3.0,2,3.0,4,0.666667
2,466315,0,1,6.7,1,4,1.0,1,1.0,6,1.000000
3,461735,1,1,6.7,1,6,0.0,3,2.0,6,1.500000
4,462669,0,1,4.0,1,3,3.0,3,6.0,6,0.500000


In [46]:
rfr.score(X,Y)

1.0

In [49]:
predictions = rfr.predict(data2)

In [50]:
Id = test_set['ID'] 

In [51]:
Solution = pd.DataFrame({'ID': Id,'Segmentation':predictions})

In [52]:
Solution.head()

,ID,Segmentation
0,458989,B
1,458994,C
2,458996,B
3,459000,C
4,459001,C


In [53]:
Solution.to_csv('Sol10.csv' , index = False)

In [44]:
from sklearn.model_selection import GridSearchCV

rfr = RandomForestClassifier()

parameters = { 'max_features':np.arange(5,10),'n_estimators':[500],'min_samples_leaf': [10,50,100,200,500]}

random_grid = GridSearchCV(rfr, parameters, cv = 5)

In [ ]:
rf = Class_Fit(clf = ensemble.RandomForestClassifier)
param_grid = {'criterion' : ['entropy', 'gini'], 'n_estimators' : [20, 40, 60, 80, 100],
               'max_features' :['sqrt', 'log2']}
rf.grid_search(parameters = param_grid, Kfold = 5)
rf.grid_fit(X = X_train, Y = Y_train)
rf.grid_predict(X_test, Y_test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dtc = DecisionTreeClassifier(random_state = 0)

In [42]:
votingC = ensemble.VotingClassifier(estimators=[('rfr', RandomForestClassifier()),('gbc', GradientBoostingClassifier())]
                                                 , voting='soft') 

In [46]:
vc = votingC.fit(X,Y)

In [47]:
votingC.score(X,Y)

0.936183955561297

In [52]:
data1.to_csv('data1.csv')

In [58]:
data1['Spending_Score'].shape[0]

7741

In [56]:
data1['X_factor'] = []

In [60]:
for i in range( data1['Spending_Score'].shape[0]):
    data1['X_factor'].append(data1['Spending_Score'][i]/data1['Family_Size'][i])

TypeError: cannot concatenate object of type "<class 'numpy.float64'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid